In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../scripts'))

In [3]:
# Load your DataFrame with the correct delimiter
df = pd.read_csv("../Data/cleaned.csv")
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827.0,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827.0,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827.0,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827.0,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827.0,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


Handle Missing Data:

In [4]:
df.columns

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice',
       'CapitalOutstanding', 'NewVehicle', 'SumInsured', 'TermFrequency',
       'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory',
       'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass',
       'StatutoryRiskType', 'TotalPremium', 'TotalClaims'],
      dtype='object')

In [5]:
df["ClaimRatio"] = df["TotalClaims"] / df["TotalPremium"]


Encode Categorical Data

In [6]:
df = pd.get_dummies(df, columns=['Gender', 'Province', 'CoverType'], drop_first=True)


Split Data into Train/Test:

In [7]:
from sklearn.model_selection import train_test_split

# Define target and features
X = df.drop(columns=['TotalPremium', 'TotalClaims', 'PolicyID', 'UnderwrittenCoverID'])  # Exclude target variables and IDs
y = df['TotalClaims']  # or 'TotalPremium' based on what you want to predict

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [12]:
df.dtypes


UnderwrittenCoverID                           int64
PolicyID                                    float64
TransactionMonth                            float64
IsVATRegistered                                bool
Citizenship                                  object
                                             ...   
CoverType_Signage and Vehicle Wraps            bool
CoverType_Standalone passenger liability       bool
CoverType_Third Party                          bool
CoverType_Trailer                              bool
CoverType_Windscreen                           bool
Length: 73, dtype: object

In [18]:
df = pd.get_dummies(df, drop_first=True)  # Automatically converts categorical columns to numerical using one-hot encoding


In [23]:
df.fillna(df.mean(), inplace=True)  # Impute missing values with the column mean


c:\Python312\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [24]:
# Convert 'TransactionMonth' to datetime first
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')

# Now, convert to the number of days since the earliest date
df['TransactionMonth'] = (df['TransactionMonth'] - df['TransactionMonth'].min()) / np.timedelta64(1, 'D')


## Model Building

Implement models

1.Linear Regression

In [27]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

# Make predictions
y_pred_lr = lr.predict(X_test)


ValueError: could not convert string to float: '2015-07-01'

2.Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)


3.XGBoost

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)


## Model Evaluation

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, rf.predict(X_test)))


Feature Importance Analysis

In [ ]:
import shap

explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test)
